In [2]:
pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [1]:
from fake_useragent import UserAgent
from os.path import join,isfile
from os import listdir
import dask
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import time
import bookquery
import sys
import os
import datetime

ua = UserAgent()#Прокси
curr_directory = os.getcwd()
new_path = join(curr_directory,"Information")
try:# Создаем базовую директорию для файлов
    os.mkdir(new_path)
except OSError:
    print ("Директория %s уже создана" % new_path)
    
def execute_query(qquery, entity_id):# Работа с запросами
    
    """Работа с запросами"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

Директория D:\Univer\Сбербанк\Work\Geography\Geography\Information уже создана


In [3]:
ua = UserAgent()
res = []
bad_requests = []# массив для ненайденных

def entities_sparqul(results):#
    
    """Обработка результата"""
    
    res = []
    for result in results:# Обработка вернувшегося запрсов
        entities = {}
        entities['entity'] = result['item']['value'].split('/')[-1]
        entities['label'] = result['item']['value'].split('/')[-1]
        showLabel = results[0]['showLabel']['value']
        res.append(entities)# Создаем массив словарей
    return pd.DataFrame(data = res),showLabel#Возвращаем Датафрейм и название сущности

def file_creation(entity_id , path, f = 0):
    
    """Создание файла со всеми сущностями"""
    
    global bad_requests
    if f > 3:
            bad_requests.append(entity_id)
            return None
    try:
        new_path = join(curr_directory,join("Information",path + "\\data"))
        if entity_id not in list(map(lambda x:x.split('\\')[-1].
                                     split('_')[-1].split('.')[0], counter_for_files(new_path))):#Проверка на наличие файла
            raw_results= execute_query(bookquery.entity_finding, entity_id)# Запрос
            result,Label = entities_sparqul(raw_results) # Обработка
            result.to_csv(join(new_path ,f'entities_{Label}_{entity_id}.csv'),
                          encoding='utf-8-sig',index = False,sep = ';')# Формирование
            print(f'Cущность {Label} закончила обрабатываться')
        else :
            print(f'Сущность {entity_id} уже обработана')
    except Exception as e:
        print(e)
        try:#Обработка внутренних ошибок
            if e.code == -1003:
                time.sleep(3)
                print('Too many requests')
                return file_creation(entity_id, f + 1)
        except Exception as e1:
            bad_requests.append(entity_id)
            return None     
        
def global_file_creation(path):
    """Функция для получения ID из файла Entites.xlsx и создания файлов со всеми сущностями"""
    curr_directory = os.getcwd()
    csv_dir = join(curr_directory,'csv/Entities.xlsx')
    new_path = join(curr_directory,join("Information",path + f"\data"))
    new_path_1 = join(curr_directory,join("Information",path))
    try:# Создаем базовую директорию для файлов
        os.mkdir(new_path_1)
    except OSError:
        print ("Директория %s уже создана" % new_path)
    try:# Создаем базовую директорию для файлов
        os.mkdir(new_path)
    except OSError:
        print ("Директория %s уже создана" % new_path)
    exl = pd.read_excel(csv_dir)
    exl = exl[exl['subclass_weight'] == int(path)]
    for i in exl['subclass']:
        res.append(dask.delayed(file_creation)(i,path))
    final_dataframe = dask.compute(*res)

global_file_creation('2')    

Директория D:\Univer\Сбербанк\Work\Geography\Geography\Information\2\data уже создана
Директория D:\Univer\Сбербанк\Work\Geography\Geography\Information\2\data уже создана
Сущность Q55488 уже обработана
Сущность Q15104915 уже обработана
Сущность Q3240715 уже обработана
Сущность Q532 уже обработана
Сущность Q835714 уже обработана
Сущность Q515 уже обработана
Сущность Q62447 уже обработана
Сущность Q831740 уже обработана
Сущность Q3957 уже обработанаСущность Q820477 уже обработана

Сущность Q39816 уже обработанаСущность Q23442 уже обработана

Сущность Q855697 уже обработана
Сущность Q523166 уже обработана
Сущность Q10354598 уже обработана
Сущность Q8514 уже обработана
Сущность Q399984 уже обработана
Сущность Q2418896 уже обработана
Сущность Q1349255 уже обработана
Сущность Q358 уже обработана
Сущность Q2989457 уже обработана
Сущность Q1248784 уже обработана
Сущность Q35657 уже обработанаСущность Q184358 уже обработана

Сущность Q8502 уже обработана
Сущность Q4421 уже обработана
Сущность 

In [12]:
def entities_sparqul_new(results):
    
    """Обработка результата"""
    
    res = []
    for result in results:# Обработка вернувшегося запрсов
        entities = {}
        entities['entity'] = result['item']['value'].split('/')[-1]
        entities['label'] = result['item']['value'].split('/')[-1]
        res.append(entities)# Создаем массив словарей
    return pd.DataFrame(data = res)#Возвращаем Датафрейм и название сущности


def file_creation_new(entity_id , Label , path, f = 0)

    """Обработка крупных файлов(свыше 100000 представителей)"""
    
    global bad_requests
    if f > 3:
            bad_requests.append(entity_id)
            return None
    try:
        new_path = join(curr_directory,join("Information",path + "\\data"))
        if entity_id not in list(map(lambda x:x.split('\\')[-1].
                                     split('_')[-1].split('.')[0], counter_for_files(new_path))):#Проверка на наличие файла
            raw_results= execute_query(bookquery.huge_entity_finding, entity_id)# Запрос
            result = entities_sparqul_new(raw_results)# Обработка
            result.to_csv(join(new_path ,f'entities_{Label}_{entity_id}.csv'),
                          encoding='utf-8-sig',index = False,sep = ';')# Формирование
        else :
            print(f'Сущность {entity_id} уже обработана')
    except Exception as e:
        print(e)
        try:#Обработка внутренних ошибок
            if e.code == -1003:
                time.sleep(3)
                print('Too many requests')
                return file_creation(entity_id, f + 1)
        except Exception as e1:
            bad_requests.append(entity_id)
            return None     
        

In [ ]:
file_creation_new('Q8502' , 'mountain' ,'2')

In [4]:
%%time
def counter_popularity(v , f = 0,entity_id = None,label  = None):
    
    """Функция возвращает популярность для каждой сущности"""
    
    if f > 25:#Если не удается нормально проработать , то записываем -1
        new_entity = {
            'entity':entity_id,
            'label':label,
            'popularity' : -1
        }
        return new_entity
    try:
        new_entity = {}
        entity_id, label = v['entity'], v['label']
        from_res =int(execute_query(bookquery.counter_from, entity_id)[0]['countOut']['value'])#Исходящие ссылки
        to_res =int(execute_query(bookquery.counter_to, entity_id)[0]['countIn']['value']) #Входящие ссылки
        new_entity = {
            'entity':entity_id,
            'label':label,
            'popularity' : from_res + to_res#общая популярность
        }
        return  new_entity
    except Exception as e:#перехватываем ошибку и слипим на 3 секунды для более корректной работы endpoint
        time.sleep(3)
        return counter_popularity(v, f + 1,entity_id, label)
        
        #Пушим в массив необработанные сущности
        
    
def determ_popul(filename,path):
    
    """Подсчет популрностей дял всех сущностей"""
    
    curr_directory = os.getcwd()
    new_path = join(curr_directory,join("Information",path + "\\data\data_with_popularity"))
    labels = filename.split('\\')[-1].split('_')#название класса
    label = labels[-2]
    entity_id = labels[-1].split('.')[0]
    if entity_id not in list(map (lambda x : x.split('_')[-3],counter_for_files(new_path))):# проверка на наличие     
        start = time.time()
        file_name = filename.split('\\')[-1].split('.')[0]
        print(f'Файл {label} начал обрабатываться {datetime.datetime.now().time()}')
        res = []
        result = []
        ids = pd.read_csv(filename, encoding='utf8',sep = ';')#читаем файл
        for k, v in ids.iterrows():#Запросы по каждой сущности
            res.append(dask.delayed(counter_popularity)(v))
#             result.append(counter_popularity(v))
        if not res:
            print(f"Файл {label} является пустым")
            return None
        result = pd.DataFrame(data = dask.compute(*res))
#         result = pd.DataFrame(data = result)
        result.sort_values(by = 'popularity',ascending = False).to_csv(
            join(new_path,file_name + '_with_popularity.csv'),
            encoding='utf-8-sig',index = False,sep = ';')# Формирование
        end = time.time()
        print(f'Формирование файла {file_name} окончено , заняло {(end-start)/60} минут\n')
    else:
        print(f'Файл {label} уже обработан')

def global_finding_popularity(path):
    
    """Проход по всем файлам в директории"""
    
    res = []
    curr_directory = os.getcwd()
    new_path = join(curr_directory,join("Information",path + "\\data\data_with_popularity"))
    path_for_finding = join(curr_directory,join("Information",path + "\\data"))
    try:
        os.mkdir(new_path)
    except OSError:
        print ("Директория %s уже создана" % new_path)
        
    for filename in counter_for_files(path_for_finding):
        determ_popul(filename,path)
    #result = dask.compute(*res)

global_finding_popularity("2")

Директория D:\Univer\Сбербанк\Work\Geography\Geography\Information\2\data\data_with_popularity уже создана
Файл aerodrome уже обработан
Файл airport уже обработан
Файл archipelago уже обработан
Файл autonomous oblast of Russia уже обработан
Файл autonomous okrug of Russia уже обработан
Файл bay уже обработан
Файл bridge уже обработан
Файл canal уже обработан
Файл canyon уже обработан
Файл cape уже обработан
Файл cave уже обработан
Файл channel уже обработан
Файл city of the United States уже обработан
Файл city town уже обработан
Файл city уже обработан
Файл constituent part of the United Kingdom уже обработан
Файл continental area and surrounding islands уже обработан
Файл continent уже обработан
Файл country уже обработан
Файл crater уже обработан
Файл desert уже обработан
Файл drainage basin уже обработан
Файл federal city of Russia уже обработан
Файл fjard уже обработан
Файл fjord уже обработан
Файл forest уже обработан
Файл fountain уже обработан
Файл geographic region уже обработ

In [13]:
def execute_query(qquery, book_id):
    
    """Обработка результата"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(book_id = book_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entity_from_sparql(results):
    
    """Обработка исходящих предикатов"""
    
    entity = {}
    for result in results:#Построчно читаем
        wdLabel, ps_Label, ps_, wd, direction = result['wdLabel']['value'], result['ps_Label']['value'], \
                        result['ps_']['value'], result['wd']['value'], result['direction']['value']
        ps = {
            'propValueID': ps_,
            'propValue': ps_Label
        }
        if wdLabel in entity:# Формируем словари для повторяющихся предикатов
                entity[wdLabel]['content'].append(ps)
        else:
            entity[wdLabel] = {
                'propID': wd,
                'direction': direction,
                'content': [ps]
            }
    return entity

def entity_to_sparql(results):
    
    """Обрабокта входящих предикатов"""
    
    entity = {}
    for result in results:
        obj, objLabel, wd, prop_label, bookLabel, direction = result['obj']['value'], result['objLabel']['value'], \
                        result['wd']['value'], result['prop_label']['value'], \
                        result['bookLabel']['value'], result['direction']['value']
        ps = {
            'propValueID': obj,
            'propValue': objLabel
        }
        
        if prop_label in entity:
                entity[prop_label]['content'].append(ps)
        else:
            entity[prop_label] = {
                'propID': wd,
                'direction': direction,
                'content': [ps]
            }
    return entity
    

def get_properties_of_book(book_id, book_name=None):
    
    """Формирование датафрейма"""
    
    results_from = execute_query(bookquery.query_from, book_id)
    results_to = execute_query(bookquery.query_to, book_id)
    
    entity_from = entity_from_sparql(results_from)
    entity_to = entity_to_sparql(results_to)
    
    entity = {**entity_from, **entity_to}# Объединяем словарь 
#     _book_name = book_name if book_name is not None else \
#                 (entity['title'][0]['ps_Label'] if 'title' in entity else book_id)
    _book_name = book_name
    
    counter, res = {}, []
    for k, v in entity.items(): 
        counter[k] = len(v['content'])
    for k, v in counter.items():# Формирвоание датафрейма
        prop = {
            'entity_id': book_id,
            'entity_name': _book_name,
            'prop_name': k,
            'prop_count':1,
            'prop_count_value': v,
            'prop_dir': entity[k]['direction']
        }
        res.append(prop)
    return pd.DataFrame(res)

def processing(v, f=0):
    
    """Обработка запроса"""
    
    if f > 20:# Если не удается , то ничего не возвращаем
            return None
    try:
        book_id, bookLabel = v['entity'], v['label']
        dataframe = get_properties_of_book(book_id, book_name=bookLabel)
        
        return  dataframe
    except Exception as e:
        time.sleep(3)
        return processing(v, f + 1)

def finding_popular(most_popular):
        
    """Обработка 5% наиболее популярных сущностей"""
        
    if len(most_popular) * 0.05 < 10:
        return list(most_popular.iloc[:10].entity)
    elif len(most_popular) * 0.05 >= 100:
        return list(most_popular.iloc[:100].entity)
    else:
        return list(most_popular.iloc[:int(len(most_popular) * 0.05)].entity)
        
def entity_analyzer(filename,path):
    
    """Создание csv-файлов"""
        
    curr_path = join(curr_directory,join("Information",path + "\\analyzing")) # Обработчик файлов
    Label = filename.split('\\')[-1].split('.')[0].split('_')[-1]# Название файла
    file_name = filename.split('\\')[-1].split('.')[0]
    label  = filename.split('\\')[-1].split('.')[0].split('_')[-2]
    if Label not in list (map(lambda x:x.split('\\')[-1].split('.')[0].split('_')[-1],counter_for_files(curr_path))):
        print(f'Файл {label} начал обрабатываться {datetime.datetime.now().time()}')
        start = time.time()
        ids = pd.read_csv(filename, encoding='utf8',sep = ';')
        stats_dfs = []
        entities = []
        res = []
        result = []
        for k, v in ids.iterrows():#Запросы по каждой сущности
#             result.append(processing(v))
            res.append(dask.delayed(processing)(v))
        if res == []:
            print(f'Файл {label} пуст')
            return None
        result = dask.compute(*res)
        errors_counter = 0
        for i in range(len(result)):#Формируем датафреймы
            try:
                stats_dfs.append(result[i])
            except Exception as e:
                errors_counter+=1
                continue
        #Создание файла со всеми предикатами для каждой сущности        
        new_path = join(curr_directory,join("Information",path + "\\data\data_with_popularity"))
        stats_df = pd.concat(stats_dfs)#Создание файлов
        stats_df.sort_values(by = 'prop_count',ascending = False).to_csv(
            curr_path + f'/analyzed_{file_name}.csv', encoding='utf-8-sig', index=False,sep = ';')

        # Создание агрегированного по сущностям файла для предикатов
        stats_aggregated = stats_df.groupby('prop_name').sum().sort_values(
            by = ['prop_count','prop_count_value'],ascending = False)
        stats_aggregated['prop_counter'] = stats_aggregated['prop_count']/(len(ids)-errors_counter) * 100
        stats_aggregated.to_csv(curr_path + f'/aggregated/{file_name}_aggregated.csv', encoding='utf-8-sig',sep = ';')

        most_popular = pd.read_csv(join(new_path,f'{file_name}_with_popularity.csv'),encoding='utf8',sep=';')
        most_popular = finding_popular(most_popular)
        def filter_func(x):
            return x['entity_id'].max() in most_popular
        #Топ 5% популярных сущностей
        popular_stats = stats_df.groupby('entity_id').filter(filter_func).groupby('prop_name').sum().sort_values(
            by = ['prop_count','prop_count_value'],ascending = False)
        popular_stats['prop_counter'] = popular_stats['prop_count']/(len(most_popular)-errors_counter) * 100
        popular_stats.to_csv(curr_path + f'/aggregated_with_popularity/{file_name}_aggregated_with_popularity.csv',
                             encoding='utf-8-sig',sep = ';')

        end = time.time()
        print(f'Файл {file_name} отработал:количество необработанных файлов {errors_counter} из {len(ids)}(успех:{(len(ids)-errors_counter)*100/(len(ids))} %), заняло {(end-start)/60} минут')
    else:
        print(f'Файл {label} уже обработан')


        
def global_analyzer(path):
    
    """Функция обхода по всем файлам директории"""
    
    res = []
    curr_directory = os.getcwd()
    
    new_path_main = join(curr_directory,join("Information",path + r"\analyzing"))
    path_for_finding = join(curr_directory,join("Information",path + r"\data"))
    try:
        os.mkdir(new_path_main)
    except OSError:
        print ("Директория %s уже создана" % new_path_main)

    new_path = join(new_path_main,'aggregated')
    try:
        os.mkdir(new_path)
    except OSError:
        print ("Директория %s уже создана" % new_path)

    new_path = join(new_path_main,'aggregated_with_popularity')
    try:
        os.mkdir(new_path)
    except OSError:
        print ("Директория %s уже создана" % new_path)
    for filename in counter_for_files(path_for_finding):
        entity_analyzer(filename,path)
        #res.append(dask.delayed( entity_analyzer)(filename,path))
    #result = dask.compute(*res)


global_analyzer("2")


Директория D:\Univer\Сбербанк\Work\Geography\Geography\Information\2\analyzing уже создана
Директория D:\Univer\Сбербанк\Work\Geography\Geography\Information\2\analyzing\aggregated уже создана
Директория D:\Univer\Сбербанк\Work\Geography\Geography\Information\2\analyzing\aggregated_with_popularity уже создана
Файл aerodrome уже обработан
Файл airport уже обработан
Файл archipelago уже обработан
Файл autonomous oblast of Russia уже обработан
Файл autonomous okrug of Russia уже обработан
Файл bay уже обработан
Файл bridge уже обработан
Файл canal уже обработан
Файл canyon уже обработан
Файл cape уже обработан
Файл cave уже обработан
Файл channel уже обработан
Файл city of the United States уже обработан
Файл city town уже обработан
Файл city уже обработан
Файл constituent part of the United Kingdom уже обработан
Файл continental area and surrounding islands уже обработан
Файл continent уже обработан
Файл country уже обработан
Файл crater уже обработан
Файл desert уже обработан
Файл drain